In [1]:
import tensorflow as tf
import tensorflow_hub as hub

2023-01-02 02:21:39.097153: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-02 02:21:39.324459: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-02 02:21:39.324510: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-02 02:21:40.374014: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

ImportError: attempted relative import with no known parent package

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

Num GPUs Available:  0


2023-01-02 02:34:01.517876: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-02 02:34:01.525256: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-02 02:34:01.525383: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-EQGCG6TT): /proc/driver/nvidia/version does not exist


### Getting the data

In [19]:
#getting the images
import os
import glob
import numpy as np
dataset_path =  os.path.abspath("../dataset")
globpath = f"{dataset_path}/**/*.jpg"
pictures = np.hstack([np.array(glob.glob(globpath, recursive = True))])
pictures[:10]

array(['/home/almudenaramirez607/Final_project/dataset/test/20/img2.jpg',
       '/home/almudenaramirez607/Final_project/dataset/test/20/d20_top_584.jpg',
       '/home/almudenaramirez607/Final_project/dataset/test/20/d20_top_624.jpg',
       '/home/almudenaramirez607/Final_project/dataset/test/20/d20_top_520.jpg',
       '/home/almudenaramirez607/Final_project/dataset/test/20/d20_top_576.jpg',
       '/home/almudenaramirez607/Final_project/dataset/test/20/d20_top_528.jpg',
       '/home/almudenaramirez607/Final_project/dataset/test/20/img1.jpg',
       '/home/almudenaramirez607/Final_project/dataset/test/20/d20_top_632.jpg',
       '/home/almudenaramirez607/Final_project/dataset/test/20/d20_top_608.jpg',
       '/home/almudenaramirez607/Final_project/dataset/test/20/d20_top_560.jpg'],
      dtype='<U80')

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

# preprocessing the data so it matches the model's expectations
# not using the ImageDataGenerator for dataset ampliation because it already has rotations and image edits
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

train_data = '/home/almudenaramirez607/Final_project/dataset/train'
test_data = '/home/almudenaramirez607/Final_project/dataset/test'

train_generator = ImageDataGenerator(preprocessing_function = preprocess_input).flow_from_directory(
    train_data, 
    target_size = (480, 480), 
    color_mode= 'grayscale', 
    batch_size = 30)
test_generator = ImageDataGenerator(preprocessing_function = preprocess_input).flow_from_directory(
    test_data, 
    target_size = (480, 480), 
    color_mode= 'grayscale', 
    batch_size = 20)


Found 3515 images belonging to 21 classes.
Found 555 images belonging to 21 classes.


### Loading Mobilenet_v2 Model

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import TensorBoard

In [21]:
# we are using default weights -> imagenet
base = MobileNetV2(weights='imagenet', include_top = False)

model = Sequential([
    base,
    GlobalAveragePooling2D(), 
    Dropout(0.2), 
    Dense(512, activation='relu'),
    Dense(210, activation='relu'), 
    Dense(21, activation='softmax')])


model.compile(
    optimizer = Adam(learning_rate = 0.001),
    loss = 'categorical_crossentropy', 
    metrics = ['accuracy'])


9406464/9406464 [==============================] - 5s 1us/step


ImportError: cannot import name 'GlobalAveragePooling' from 'tensorflow.keras.layers' (/home/almudenaramirez607/miniconda3/envs/tf/lib/python3.9/site-packages/keras/api/_v2/keras/layers/__init__.py)

In [23]:
tensorboarddata = TensorBoard(log_dir='logs/mobilenet_board')

In [ ]:

%load_ext tensorboard

%tensorboard --logdir logs/mobilenet_board